<a href="https://colab.research.google.com/github/Rogerio5/Dimensionality-Reduction-in-Images-for-Neural-Networks-/blob/main/Dimensionality_Reduction_in_Images_for_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔧 ETAPA 1: Upload da imagem
from google.colab import files

print("📤 Faça o upload da imagem PPM (formato P3 ou P6)")
uploaded = files.upload()
nome_arquivo = list(uploaded.keys())[0]
print(f"✅ Arquivo recebido: {nome_arquivo}")



In [ ]:
# 📦 ETAPA 2: Funções utilitárias

# Leitura de imagem PPM (formato P3)
def ler_ppm_p3(caminho):
    with open(caminho, 'r') as f:
        linhas = f.readlines()

    linhas = [linha.strip() for linha in linhas if not linha.startswith('#') and linha.strip()]
    if linhas[0] != 'P3':
        raise ValueError('Apenas imagens PPM no formato P3 são suportadas.')

    largura, altura = map(int, linhas[1].split())
    max_valor = int(linhas[2])

    pixels = list(map(int, ' '.join(linhas[3:]).split()))
    imagem = []
    i = 0
    for y in range(altura):
        linha = []
        for x in range(largura):
            r, g, b = pixels[i], pixels[i+1], pixels[i+2]
            linha.append((r, g, b))
            i += 3
        imagem.append(linha)

    return imagem, largura, altura, max_valor

# Leitura de imagem PPM (formato P6)
def ler_ppm_p6(caminho):
    with open(caminho, 'rb') as f:
        header = f.readline().strip()
        if header != b'P6':
            raise ValueError('Apenas imagens PPM no formato P6 são suportadas.')

        def ler_linha_valida():
            linha = f.readline()
            while linha.startswith(b'#'):
                linha = f.readline()
            return linha

        dimensoes = ler_linha_valida().strip()
        largura, altura = map(int, dimensoes.split())
        max_valor = int(ler_linha_valida().strip())

        dados = f.read()
        imagem = []
        i = 0
        for y in range(altura):
            linha = []
            for x in range(largura):
                r, g, b = dados[i], dados[i+1], dados[i+2]
                linha.append((r, g, b))
                i += 3
            imagem.append(linha)

        return imagem, largura, altura, max_valor

# Detecta e escolhe o leitor correto
def detectar_e_ler_ppm(caminho):
    with open(caminho, 'rb') as f:
        assinatura = f.readline().strip()

    if assinatura == b'P3':
        return ler_ppm_p3(caminho)
    elif assinatura == b'P6':
        return ler_ppm_p6(caminho)
    else:
        raise ValueError(f'Formato não reconhecido: {assinatura}')

# Conversão para cinza
def converter_para_cinza(imagem):
    return [[(cinza := int(0.299*r + 0.587*g + 0.114*b),)*3 for r, g, b in linha] for linha in imagem]

# Binarização
def converter_para_binaria(imagem_cinza, limiar=127):
    return [[(255, 255, 255) if r > limiar else (0, 0, 0) for r, _, _ in linha] for linha in imagem_cinza]

# Salvamento em PPM (formato P3)
def salvar_ppm_p3(imagem, largura, altura, caminho_saida):
    with open(caminho_saida, 'w') as f:
        f.write('P3\n')
        f.write(f'{largura} {altura}\n')
        f.write('255\n')
        for linha in imagem:
            f.write(' '.join(f'{r} {g} {b}' for r, g, b in linha) + '\n')


In [ ]:
# 🖼️ ETAPA 3: Processamento da imagem

# Leitura automática
imagem_colorida, largura, altura, max_valor = detectar_e_ler_ppm(nome_arquivo)

# 🔽 Reduz dimensões se necessário (para manter <1MB)
def reduzir_resolucao(imagem, largura, altura, fator=2):
    nova_largura = largura // fator
    nova_altura = altura // fator
    imagem_redimensionada = [
        [imagem[y*fator][x*fator] for x in range(nova_largura)]
        for y in range(nova_altura)
    ]
    return imagem_redimensionada, nova_largura, nova_altura

if largura * altura > 300_000:  # Limite aproximado para <1MB
    imagem_colorida, largura, altura = reduzir_resolucao(imagem_colorida, largura, altura)

# Conversão
imagem_cinza = converter_para_cinza(imagem_colorida)
imagem_binaria = converter_para_binaria(imagem_cinza)

# Salvamento
salvar_ppm_p3(imagem_colorida, largura, altura, 'imagem_colorida.ppm')
salvar_ppm_p3(imagem_cinza, largura, altura, 'imagem_cinza.ppm')
salvar_ppm_p3(imagem_binaria, largura, altura, 'imagem_binaria.ppm')

print("✅ Imagens geradas com sucesso!")


In [ ]:
# 📥 ETAPA 4: Download dos arquivos
from google.colab import files

files.download('imagem_colorida.ppm')
files.download('imagem_cinza.ppm')
files.download('imagem_binaria.ppm')


In [ ]:
# 🌐 ETAPA 5: Visualização em HTML
from IPython.display import display, HTML

def gerar_html_imagem(imagem, titulo):
    html = f"<div style='margin:10px;text-align:center;'>"
    html += f"<h4>{titulo}</h4><div style='line-height:0;'>"
    for linha in imagem:
        for r, g, b in linha:
            html += f"<span style='display:inline-block;width:1px;height:1px;background-color:rgb({r},{g},{b});'></span>"
        html += "<br>"
    html += "</div></div>"
    return html

# Gerar HTML das três imagens
html_colorida = gerar_html_imagem(imagem_colorida, 'Colorida')
html_cinza = gerar_html_imagem(imagem_cinza, 'Cinza')
html_binaria = gerar_html_imagem(imagem_binaria, 'Binária')

# Exibir lado a lado
html_final = f"""
<div style='display:flex;flex-direction:row;justify-content:center;gap:20px;'>
  {html_colorida}
  {html_cinza}
  {html_binaria}
</div>
"""

display(HTML(html_final))
